In [6]:
import pandas as pd
import requests
from itertools import chain
from googlesearch import search
import os
import simplemma as splm
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


#try:
#    unis = pd.read_excel('unis_output_cor.xlsx','Sheet1')
#    url_link = 'https://infoetudes.com/liste-adresses-et-contacts-des-universites-ecoles-de-formations-et-instituts-du-senegal/'
#    r = requests.get(url_link,headers ={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'})
#    unis = pd.read_html(r.text)[0]
#    unis = unis.transpose()
#    unis.drop(0,axis=1,inplace=True)
#    unis.set_index(1,inplace=True)
#    unis = unis.transpose()
#    unis.reset_index(inplace=True)
#    unis.drop('index',axis=1,inplace=True)
#    unis.columns = ['nom', 'adresse', 'details[1]', 'details']
#    unis.dropna(thresh=3,inplace=True)
#    unis.drop('details[1]',axis=1,inplace=True)
#    unis.to_excel('unis_output.xlsx')
#except:
#    pass
unis = pd.read_excel('unis.xlsx','Sheet1')

unis.drop('Unnamed: 0',axis=1,inplace=True)
unis["nom"] = unis["nom"]
unis["adresse"] = unis["adresse"]
unis["details"] = unis["details"]
unis.fillna('',inplace=True)
keyword_list = []
for ndet in unis['details'].str.lower():
    ndet = ndet.replace(",","")
    ndet = ndet.replace(".","")
    ndet = ndet.replace("…","")
    ndet = ndet.replace("’", " ")
    ndet = ndet.replace("/", " ")
    for i in set(ndet.split()):
        if len(i) > 2:
            keyword_list.append(i)
new_keyword_list=[]
for i in keyword_list:
    if i[-1] == 's': i = i[:-1]
    if len(i) > 2:
        new_keyword_list.append(i)


langdata = splm.load_data('fr')
stop_words = set(stopwords.words('french'))

unvect = []
for i in unis['details']:
    word_tokens = word_tokenize(i)
    filtered_sentence = []
    for w in word_tokens:
        if w not in stop_words and len(w)>2:
            filtered_sentence.append(splm.lemmatize(w, langdata))

    sent = ' '.join(filtered_sentence)
    unvect.append(sent)

unis['unvectored'] = unvect
unis['unvectored'].mask(unis['unvectored'] == '',inplace=True)

unique_words = list(chain(*[unvect[i].split(" ") for i in range(len(unvect))]))

#print(unique_words)




In [7]:
stop_words

{'ai',
 'aie',
 'aient',
 'aies',
 'ait',
 'as',
 'au',
 'aura',
 'aurai',
 'auraient',
 'aurais',
 'aurait',
 'auras',
 'aurez',
 'auriez',
 'aurions',
 'aurons',
 'auront',
 'aux',
 'avaient',
 'avais',
 'avait',
 'avec',
 'avez',
 'aviez',
 'avions',
 'avons',
 'ayant',
 'ayante',
 'ayantes',
 'ayants',
 'ayez',
 'ayons',
 'c',
 'ce',
 'ces',
 'd',
 'dans',
 'de',
 'des',
 'du',
 'elle',
 'en',
 'es',
 'est',
 'et',
 'eu',
 'eue',
 'eues',
 'eurent',
 'eus',
 'eusse',
 'eussent',
 'eusses',
 'eussiez',
 'eussions',
 'eut',
 'eux',
 'eûmes',
 'eût',
 'eûtes',
 'furent',
 'fus',
 'fusse',
 'fussent',
 'fusses',
 'fussiez',
 'fussions',
 'fut',
 'fûmes',
 'fût',
 'fûtes',
 'il',
 'ils',
 'j',
 'je',
 'l',
 'la',
 'le',
 'les',
 'leur',
 'lui',
 'm',
 'ma',
 'mais',
 'me',
 'mes',
 'moi',
 'mon',
 'même',
 'n',
 'ne',
 'nos',
 'notre',
 'nous',
 'on',
 'ont',
 'ou',
 'par',
 'pas',
 'pour',
 'qu',
 'que',
 'qui',
 's',
 'sa',
 'se',
 'sera',
 'serai',
 'seraient',
 'serais',
 'serait',


In [8]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [9]:
keyword_series = pd.Series(unique_words)
keyword_series.value_counts().to_csv('keywordlist')
keyword_series = pd.read_csv('keywordlist')

In [11]:
keyword_series = keyword_series[(keyword_series['0']>1) & (len(keyword_series['Unnamed: 0']) > 5)]
print(keyword_series)

             Unnamed: 0   0
0               gestion  47
1               science  30
2            management  30
3          informatique  28
4         international  22
5             marketing  19
6             formation  18
7              commerce  16
8         communication  14
9             supérieur  13
10              affaire  13
11             appliqué  12
12                droit  11
13               réseau  11
14           entreprise  11
15          technologie  11
16         comptabilité  11
17            juridique  10
18                génie  10
19              finance  10
20           économique   9
21           logistique   9
22                école   9
23            assurance   9
24             institut   8
25               humain   8
26            transport   8
27               centre   7
28           ingénierie   7
29               banque   7
30              licence   7
31              diplôme   6
32        environnement   6
33        développement   6
34         enseignem